In [15]:
using StatsBase

#### Funções auxiliares

In [16]:
# mínimo entre a e b
function min2(a::Int, b::Int)
    a < b ? a : b
end

# máximo entre a e b
function max2(a::Int, b::Int)
    a > b ? a : b
end

# segundo menor elemento de um vetor
# slides aula 6, pág 22
# complexidade theta(n)
# complexidade theta(4) = constante para esta aplicação
function segundo_menor(v::Vector{Int})
    menor    = min2(v[1], v[2])
    segmenor = max2(v[1], v[2])
    n = size(v)[1]
    for k in 3:n
        if v[k] < menor
            segmenor = menor
            menor = v[k]
        elseif v[k] < segmenor
            segmenor = v[k]
        end
    end
    return(segmenor)
end

# devolve as coordenadas da mediana do vetor A[p...r] em uma tupla
function medianCoordinates(p::Int, r::Int)
    (floor(Int, (p+r)/2), ceil(Int, (p+r)/2))
end

medianCoordinates (generic function with 1 method)

In [17]:
# A[i] <-> A[j]
# theta(1)
function swap(A::Vector{Int}, i::Int, j::Int)
    temp = A[i]
    A[i] = A[j]
    A[j] = temp
end

# CLRS page 124
# theta(n)
function partition(A::Vector{Int}, p::Int, r::Int)
    x = A[r]                # x é o pivô, último elemento entre p:r
    i = p-1                 # i inicializa como imediatemente anterior a p
    for j in p : r-1        # varredura entre p até antes do pivô
        if A[j] <= x        # compara A[j] com o pivô
            i += 1          # avança i, evita problema com inicialização i=0 (quando p=1)
            swap(A, i, j)   # A[i] <-> A[j]
        end
    end
    swap(A, i+1, r)         # A[i+1] <-> A[r]   swap do pivô, que não entrou no loop for

    return(i+1)
end

# CLRS page 130
function randomized_partition(A::Vector{Int}, p::Int, r::Int)
    i = rand(p:r)            # sorteia elemento i entre p:r
    swap(A, r, i)            # A[r] <-> A[i]
    partition(A, p, r)       # retorna o pivô q
end

function medianValue(A::Vector{Int})
    n = size(A)[1]
    medianPosition = floor(Int, n/2)
    res = sort(A)[medianPosition]
end

# theta(n)
function median_partition(A::Vector{Int}, p::Int, r::Int)
    local mp
    for i in p:r
        if A[i] == medianValue(A[p:r])
            mp = i
            break
        end
    end
    swap(A, r, mp)                      # swap da mediana com o último elemento
    partition(A, p, r)
end

median_partition (generic function with 1 method)

#### Função principal

In [18]:
# Base: Randomized Select
# CLRS page 157
function kmenor(A, p, r, k)
    if r == p                           # base da recursão
        res = A[p]
        return(res)
    end
    
    q = median_partition(A, p, r)
    len = q-p+1

    if k == len                         # k == q
        res = A[q]
        return(res)
    elseif k < len                      # k entre p:q-1
        res = kmenor(A, p, q-1, k)
        return(res)
    elseif k > len                      # k entre q+1:r
        res = kmenor(A, q+1, r, k-len)
        return(res)
    end
end

kmenor (generic function with 1 method)

#### Verificação do algoritmo

In [19]:
# funçao referência para o cálculo do k menor
function ref(X::Vector{Int}, k::Int)
    v = sort(X)
    res = v[k]
end

ref (generic function with 1 method)

In [20]:
# verificação do cálculo do k-ésimo contra a referência
nTestes = 1000000
ok = []
for i in 1 : nTestes
    N = rand(1:10)
    A = sample(1:2N, N, replace = false)
    k = rand(1:N)

    km = kmenor(A, 1, N, k)
    push!( ok, km == ref(A, k) )
end
println("testes aprovados: ", sum(ok) == nTestes)


testes aprovados: true
